In [1]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
img_width, img_height = 224, 224
train_data_dir = './data/train'
#train_data_dir = os.path.join(os.getcwd(), 'test')           
validation_data_dir = './data/train'
nb_train_samples = 1000
nb_validation_samples = 1000
epochs = 5
batch_size = 200
input_shape = (img_height, img_width, 3)
num_classes = 2 # 類別個數

In [3]:
model_vgg16_conv = VGG16(weights='imagenet', include_top=False)
model_vgg16_conv.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [4]:
#Create your own input format (here 224x224x3)
input = Input(shape=(img_width, img_height,3), name='image_input')

In [5]:
#Use the generated model 
output_vgg16_conv = model_vgg16_conv(input)

In [6]:
#Add the fully-connected layers 
x = Flatten(name='flatten')(output_vgg16_conv)
#x = Dense(4096, activation='relu', name='fc1')(x)
x = Dense(256, activation='relu', name='fc2')(x)
x = Dense(num_classes, activation='softmax', name='predictions')(x)
#x = Dense(2, activation='sigmoid')(x)

In [7]:
#Create your own model 
model = Model(inputs=input, outputs=x)

In [8]:
model.compile(loss='categorical_crossentropy', #sparse_categorical_crossentropy',
              optimizer=optimizers.SGD(learning_rate=1e-4, momentum=0.9),
              metrics=['accuracy'])


In [12]:
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image_dataset_from_directory
# label_mode='categorical' 對應 model.compile(loss='categorical_crossentropy'
# label_mode='int' 對應 model.compile(loss='sparse_categorical_crossentropy'

import tensorflow as tf

ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_data_dir, validation_split=0.2, image_size= (img_width, img_height),
    seed=0, subset='training', batch_size= batch_size, label_mode='categorical')

ds_val = tf.keras.preprocessing.image_dataset_from_directory(
    train_data_dir, validation_split=0.2, image_size= (img_width, img_height),
    seed=0, subset='validation', batch_size= batch_size, label_mode='categorical')


ds = ds.repeat().cache().prefetch(tf.data.AUTOTUNE)

# data_augmentation = ImageDataGenerator(
#     # rescale=1. / 255,
#     rotation_range = 0.2, # 旋轉
#     shear_range=0.2, # 裁剪
#     zoom_range=0.2,  # 放大/縮小
#     horizontal_flip=False) # 水平翻轉

# RandomFlip("horizontal")：水平翻轉
# RandomRotation(0.1)：旋轉 0.1 比例 
from tensorflow.keras import layers 
data_augmentation = tf.keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.1),
    ]
)

augmented_train_ds = ds.map(
  lambda x, y: (data_augmentation(x, training=True), y))

train_history = model.fit(
    augmented_train_ds,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=ds_val,
    validation_steps=nb_validation_samples // batch_size)

Found 45 files belonging to 2 classes.
Using 36 files for training.
Found 45 files belonging to 2 classes.
Using 9 files for validation.
Epoch 1/5
5/5 [==============================] - 8s 2s/step - loss: 0.0081 - accuracy: 0.9944 - val_loss: 1.1938 - val_accuracy: 0.7778
Epoch 2/5
5/5 [==============================] - 7s 1s/step - loss: 0.0013 - accuracy: 1.0000
Epoch 3/5
5/5 [==============================] - 8s 2s/step - loss: 0.0020 - accuracy: 1.0000
Epoch 4/5
1/5 [=====>........................] - ETA: 6s - loss: 0.0100 - accuracy: 1.0000

ResourceExhaustedError:  OOM when allocating tensor with shape[36,64,224,224] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node gradient_tape/model/vgg16/block1_conv2/Conv2D/Conv2DBackpropInput (defined at <ipython-input-9-0b1a960984fa>:38) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_1913]

Function call stack:
train_function


In [ ]:
img_path = './data/train/cats/cat.0.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x1 = image.img_to_array(img)
x1 = np.expand_dims(x1, axis=0)
x1 = preprocess_input(x1)

labels=['cat', 'dog']
predictions = model.predict(x1)
print('predict class:', labels[np.argmax(predictions)])
print('predict:', predictions.ravel())
